In [2]:
import subprocess
from pathlib import Path
import os
import time

In [4]:
job_templet = """#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --output=slurm_logs/%x_%A.out
#SBATCH --error=slurm_logs/%x_%A.err
#SBATCH --partition=C9654
#SBATCH --nodelist=c3
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=64
#SBATCH --mem-per-gpu=100G
#SBATCH --gres=gpu:1
export PATH=/opt/share/miniconda3/envs/mofmthnn/bin/:$PATH
export LD_LIBRARY_PATH=/opt/share/miniconda3/envs/mofmthnn/lib/:$LD_LIBRARY_PATH

srun python -u {py_executor} --progress_bar --task_cfg {task_config} --model_cfg {model_config}
""".strip()

def run_slurm_job(work_dir, executor="sbatch", script_name="run"):
    work_dir = Path(work_dir)
    # Create a script to run the job
    process = subprocess.Popen(
        f"{executor} {work_dir/script_name}",
        # [executor, str(work_dir/'run'), "&"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        shell=True,                                                                                                                         
        env=os.environ.copy(),
        cwd=str(work_dir)
    )
    return process

## Training original CGCNN model

In [6]:
work_dir = Path("./CGCNN_MT").absolute()

task_configs = [
    # "tsd_ssd",
    "tsd_ssd_ws24",
    # "tsd_ssd_ws24_water",
    # "tsd_ssd_ws24_water_water4",
    # "tsd_ssd_ws24_water",
    # "ssd_ws24",
    # "ws24",
    "tsd",
    "ssd",
    "ws24_water",
    "ws24_water4",
    "ws24_acid",
    "ws24_base",
    "ws24_boiling"
                    ]
model_configs = [
    # "att_cgcnn",
    # "cgcnn",
    "cgcnn_raw",
    # "fcnn",
    # "att_fcnn",
    # "cgcnn_uni_atom"
]
script_name = "run_slurm.sh"
py_executor = "hyperopt.py"
# py_executor = "main.py"
model_conf = {
            'batch_size': 32,
            'max_epochs': 500, 
            'max_graph_len': 200,
            'atom_fea_len': 256,
            'extra_fea_len': 16,
            'h_fea_len': 128,
            'n_conv': 6,
            'n_h': 4,
            'dropout_prob': 0.5,
            'use_extra_fea': False,
            'use_cell_params': False,
            'atom_layer_norm': False,
            'loss_aggregation': "fixed_weight_sum",   # fixed_weight_sum, dwa, sum, sample_weight_sum, trainable_weight_sum
            'dl_sampler': 'random',
            'task_att_type': 'none',
            'augment': False,
            'lr': 0.001,
            'lr_mult': 10,
            'group_lr': False,
            'optim_config': "fine",  # fine or coarse
            'auto_lr_bs_find': False, 
            'patience': 50,
            'task_norm': False,
            'log_dir': "logs",
            'optuna_name': "optuna",
            }

for task_config in task_configs:
    for model_config in model_configs:
        job_name = f"{task_config.replace('_config', '')}_{model_config.replace('_config', '')}"
        if py_executor == "hyperopt.py":
            job_name = "opt_" + job_name
            # job_templet_ = job_templet + " --pruning"
            job_templet_ = job_templet
        else:
            job_templet_ = job_templet
        job_script = job_templet_.format(job_name=job_name, 
                                        task_config=task_config, 
                                        model_config=model_config,
                                        py_executor=py_executor
                                        )
        
        for key, value in model_conf.items():
            if isinstance(value, bool):
                if value:
                    job_script += f" --{key}"
                continue
            job_script += f" --{key} {value}"
        with open(work_dir/script_name, "w") as f:
            f.write(job_script)
        process = run_slurm_job(work_dir, executor="sbatch", script_name=script_name)
        ## get the output of the job
        while True:
            output = process.stdout.readline()
            if output == b'' and process.poll() is not None:
                break
            if output:
                print(output.decode().strip())
        print(f"Submitted job {job_name} with PID {process.pid}")
        time.sleep(1)

Submitted batch job 199555
Submitted job opt_tsd_ssd_ws24_cgcnn_raw with PID 1736896


## Training refined CGCNN model (MOFSNN)

In [ ]:
# ...existing code...

from pathlib import Path
import subprocess
import os
import time
import multiprocessing

work_dir = Path("./CGCNN_MT").absolute()

task_configs = [
    "tsd_ssd_ws24",
]

model_configs = [
    "att_cgcnn",
]

py_executor = "hyperopt.py"

model_conf = {
    'batch_size': 256,
    'max_epochs': 500,
    'max_graph_len': 200,
    'atom_fea_len': 256,
    'extra_fea_len': 16,
    'h_fea_len': 128,
    'n_conv': 6,
    'n_h': 4,
    'dropout_prob': 0.5,
    'use_extra_fea': False,
    'use_cell_params': True,
    'atom_layer_norm': True,
    'loss_aggregation': "fixed_weight_sum",
    'dl_sampler': 'random',
    'task_att_type': 'self',
    'augment': False,
    'lr': 0.001,
    'lr_mult': 10,
    'group_lr': True,
    'optim_config': "fine",
    'auto_lr_bs_find': False,
    'patience': 50,
    'task_norm': True,
    'log_dir': "logs",
    'optuna_name': "optuna",
}

# Use many workers to exploit CPUs (but not all, keep a couple free)
num_workers = max(10, multiprocessing.cpu_count() - 2)

for task_config in task_configs:
    for model_config in model_configs:
        job_name = f"{task_config.replace('_config', '')}_{model_config.replace('_config', '')}"
        if py_executor == "hyperopt.py":
            job_name = "opt_" + job_name

        # Build the command line: python hyperopt.py --progress_bar ...
        cmd = [
            "python",
            "-u",
            py_executor,
            "--progress_bar",
            "--task_cfg",
            task_config,
            "--model_cfg",
            model_config,
            "--num_workers",
            str(num_workers),
        ]

        # Append model_conf as CLI args
        for key, value in model_conf.items():
            cli_key = f"--{key}"
            if isinstance(value, bool):
                if value:
                    cmd.append(cli_key)
                continue
            cmd.extend([cli_key, str(value)])

        print("Launching:", " ".join(cmd))
        process = subprocess.Popen(
            cmd,
            cwd=str(work_dir),
            env=os.environ.copy(),
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
        )

        # Stream output live (training logs will appear here)
        while True:
            line = process.stdout.readline()
            if not line and process.poll() is not None:
                break
            if line:
                print(line.decode().rstrip())

        print(f"Training completed with return code: {process.returncode}")
        time.sleep(1)

Launching: python -u hyperopt.py --progress_bar --task_cfg tsd_ssd_ws24 --model_cfg att_cgcnn --num_workers 14 --batch_size 128 --max_epochs 500 --max_graph_len 200 --atom_fea_len 256 --extra_fea_len 16 --h_fea_len 128 --n_conv 6 --n_h 4 --dropout_prob 0.5 --use_cell_params --atom_layer_norm --loss_aggregation fixed_weight_sum --dl_sampler random --task_att_type self --lr 0.001 --lr_mult 10 --group_lr --optim_config fine --patience 50 --task_norm --log_dir logs --optuna_name optuna
/home/dharunkraja/miniconda3/envs/mofsnn/lib/python3.9/site-packages/lightning_fabric/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)
/home/dharunkraja/miniconda3/envs/mofsnn/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: F

KeyboardInterrupt: 